### Team Member Details
- **Group Name:** Data Science Bank Marketers
- **Members:**
  - Amr Hacoglu – amr.hacoglu@gmail.com - Turkey - University of Karabuk - Data Science
  - Ha My Pham – mpham25@wooster.edu - US – College of Wooster – Data Science


### Problem Description
ABC Bank aims to develop a machine learning model to predict whether a customer will subscribe to a term deposit product. This model will help the bank focus its marketing efforts on customers with a higher likelihood of purchasing the product, thereby optimizing resource allocation and reducing marketing costs.


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
from statistics import mean
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import LabelEncoder

In [2]:
df = pd.read_csv('/Users/admin/Desktop/DataGlacier/Dataset/bank-full.csv', sep=';')
df.head()

df2=pd.read_csv('/Users/admin/Desktop/DataGlacier/Dataset/bank-additional-full.csv', sep = ';')
df2.head()

FileNotFoundError: [Errno 2] No such file or directory: '/Users/admin/Desktop/DataGlacier/Dataset/bank-full.csv'

In [ ]:
print(df.info())

In [ ]:
print(df2.info())

In [ ]:
print(df2.duplicated())

In [3]:
# Method 1: Mean imputation for numeric columns
numeric_columns = df2.select_dtypes(include=[np.number]).columns
df2[numeric_columns] = df2[numeric_columns].fillna(df2[numeric_columns].mean())

# Method 2: Mode imputation for categorical columns
categorical_columns = df2.select_dtypes(include=['object']).columns
df2[categorical_columns] = df2[categorical_columns].fillna(df2[categorical_columns].mode().iloc[0])

print(df2.isnull().sum())

NameError: name 'df2' is not defined

In [ ]:
# Outlier detection and handling using IQR method
def handle_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    df[column] = df[column].clip(lower_bound, upper_bound)

numeric_columns = df2.select_dtypes(include=[np.number]).columns
for column in numeric_columns:
    handle_outliers(df2, column)

print("Outliers handled for numeric columns")

In [ ]:
# Check for and remove duplicate records
duplicates = df2.duplicated().sum()
print(f"Number of duplicate records: {duplicates}")
df2.drop_duplicates(inplace=True)

# Additional data quality checks
print(df2.describe())
print(df2.info())

In [ ]:
job=df2['job'].value_counts() 
print(job)
marriage=df2['marital'].value_counts()
print(marriage)
education=df2['education'].value_counts()
print(education)
loan=df2['loan'].value_counts()
print(loan)
poutcome=df2['poutcome'].value_counts()
print(poutcome)
outcome=df2['y'].value_counts()
print(outcome)

In [ ]:
freq_table = pd.crosstab(df2['poutcome'], df2['y']) 
freq_table

In [ ]:
plt.hist(df2['age'])
plt.show()

In [ ]:
plt.hist(df2['pdays'])
plt.show()

In [ ]:
plt.hist(df2['previous'])
plt.show()

In [ ]:
df2['previous'].describe()

In [ ]:
df2['campaign'].describe()

In [ ]:
df2['duration'].describe()

In [ ]:
x=df2.iloc[:, :20]
x

In [ ]:

le = LabelEncoder()
df2['month'] = le.fit_transform(df2['month'])
df2['day_of_week'] = le.fit_transform(df2['day_of_week'])
df2['poutcome'] = le.fit_transform(df2['poutcome'])
df2['job'] = le.fit_transform(df2['job'])
df2['marital'] = le.fit_transform(df2['marital'])
df2['education'] = le.fit_transform(df2['education'])
df2['default'] = le.fit_transform(df2['default'])
df2['housing'] = le.fit_transform(df2['housing'])
df2['loan'] = le.fit_transform(df2['loan'])
df2['contact'] = le.fit_transform(df2['contact'])
df2['y'] = le.fit_transform(df2['y'])
x=df2.iloc[:, :20]
x
y=df2['y']
y

In [ ]:
x.describe()
x.info()

In [ ]:
#Use SMOTE to oversample the minority class
oversample = SMOTE()
over_X, over_y = oversample.fit_resample(x, y)
over_X_train, over_X_test, over_y_train, over_y_test = train_test_split(over_X, over_y, test_size=0.1, stratify=over_y)

In [ ]:
#Build SMOTE SRF model
SMOTE_SRF = RandomForestClassifier(n_estimators=100, random_state=0)
#Create Stratified K-fold cross validation
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
scoring = ('f1', 'recall', 'precision')
#Evaluate SMOTE SRF model
scores = cross_validate(SMOTE_SRF, over_X, over_y, scoring=scoring, cv=cv)
#Get average evaluation metrics
print('Mean f1: %.3f' % mean(scores['test_f1']))
print('Mean recall: %.3f' % mean(scores['test_recall']))
print('Mean precision: %.3f' % mean(scores['test_precision']))

In [ ]:
#Randomly spilt dataset to test and train set
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.1, stratify=y)
#Train SMOTE SRF
SMOTE_SRF.fit(over_X_train, over_y_train)
#SMOTE SRF prediction result
y_pred = SMOTE_SRF.predict(X_test)
# Create and display confusion matrix
cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Will Not Buy', 'Will Buy'])
disp.plot(cmap='Greens')
plt.title('SMOTE + Standard Random Forest Confusion Matrix')
plt.show()

In [ ]:
#from sklearn.tree import plot_tree

# Assuming your RandomForestClassifier is named 'SMOTE_SRF'
# Extract a single tree from the forest (e.g., the first tree)
tree = SMOTE_SRF.estimators_[0]

# Plot the tree
#plt.figure(figsize=(20,10))
#plot_tree(tree, filled=True, feature_names=x.columns, class_names=['Will Not Buy', 'Will Buy'], rounded=True)
#plt.title('Decision Tree from RandomForestClassifier')
#plt.show()

In [ ]:
# Extract feature importances
feature_importances = SMOTE_SRF.feature_importances_

# Create a DataFrame for better visualization
features = pd.DataFrame({
    'Feature': x.columns,
    'Importance': feature_importances
})

# Sort the DataFrame by importance
features = features.sort_values(by='Importance', ascending=False)

# Plot the feature importances
plt.figure(figsize=(12, 8))
plt.barh(features['Feature'], features['Importance'])
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.title('Feature Importances from RandomForestClassifier')
plt.gca().invert_yaxis()
plt.show()

# Display the feature importances DataFrame
features

In [ ]:
features=df2[['duration','euribor3m','nr.employed','cons.conf.idx','cons.price.idx']]
print(features)
Model = RandomForestClassifier(n_estimators=100, random_state=0, max_depth=3)
Model.fit(features,y)

In [ ]:
from sklearn.tree import plot_tree
# Extract a single tree from the forest (e.g., the first tree)
tree = Model.estimators_[0]

# Plot the tree
plt.figure(figsize=(20,10))
plot_tree(tree, filled=True, feature_names=features.columns, class_names=['Will Not Buy', 'Will Buy'], rounded=True)
plt.title('Decision Tree from RandomForestClassifier')
plt.show()